<center>
  <font size="7">Déploiement du modèle</font><br>
  <font size="5">Projet 7 - Implémentez un modèle de scoring</font>
</center>
<div align="right">
  <font size="4"><i>par Jean Vallée</i></font>
</div>

<hr size=5>

# Pré-requis

In [2]:
model_path_staging = './staging_model'
ip_server = '13.92.86.145'

In [3]:
! pip install -r $model_path_staging/requirements.txt

In [4]:
import mlflow
import numpy as np
import pandas as pd

# Serveur de Staging

## Chargement du modèle

In [5]:
staging_model = mlflow.pyfunc.load_model(model_path_staging)
staging_model

2024/06/04 13:18:08 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.8.19`, differs from the version of Python that is currently running, `Python 3.10.12`, and may be incompatible


mlflow.pyfunc.loaded_model:
  flavor: mlflow.sklearn

### Vérification de la signature

La signature du modèle chargé indique les valeurs valides à lui fournir

Attributs

In [18]:
li_features = staging_model.metadata.get_input_schema().input_names()
li_features

['CODE_GENDER_M',
 'NAME_CONTRACT_TYPE_Cash_loans',
 'NAME_EDUCATION_TYPE_Lower_secondary',
 'EXT_SOURCE_3',
 'NAME_EDUCATION_TYPE_Higher_education',
 'NAME_EDUCATION_TYPE_Secondary_or_secondary_special']

Cible

In [7]:
staging_model.metadata.get_output_schema().input_names

<bound method Schema.input_names of ['TARGET': long (required)]>

### Vérification de prédiction

On prédit la cible pour une observation

In [8]:
df_observation = pd.DataFrame(np.array([[1, 1, 0, 1.0, 1, 0]]).astype('int'), columns=li_features)
df_observation['EXT_SOURCE_3'] = df_observation['EXT_SOURCE_3'].astype('double')  # double~float64
print('Predicted target value =', str(staging_model.predict(df_observation)))

Predicted target value = [0]


### Préparation des tests unitaires
Dans ce notebook,
- les tests unitaires ne s'y dérouleront pas
- les données destinées aux tests sont sauvegardées dans des fichiers 

In [12]:
dir_test_data = '../test_api/data/'
! mkdir -p $dir_test_data

#### Liste d'attributs
Ce fichier sera utilisé par la page _form.html_

In [79]:
str_li_features = str(li_features)
print(str_li_features)
with open(dir_test_data + '/li_features.txt', "w") as file_form:
    print(str_li_features, file=file_form)

['CODE_GENDER_M', 'NAME_CONTRACT_TYPE_Cash_loans', 'NAME_EDUCATION_TYPE_Lower_secondary', 'EXT_SOURCE_3', 'NAME_EDUCATION_TYPE_Higher_education', 'NAME_EDUCATION_TYPE_Secondary_or_secondary_special']


#### Liste de types
Ce fichier sera utilisé par la page _form.html_

In [13]:
li_types = staging_model.metadata.get_input_schema().input_types()
str_li_types = str(li_types)
print(str_li_types)
with open(dir_test_data + '/li_types.txt', "w") as file_form:
    print(str_li_types, file=file_form)

[long, long, long, double, long, long]


#### Dictionnaire simple d'observations
Ce fichier sera utilisé par le test unitaire de connexion du script _test_1.py_

In [80]:
str_features_values = df_observation.to_json(orient='records')
print(str_features_values)
with open(dir_test_data + '/dict_X_single.json', 'w') as file_object:
    print(str_features_values, file=file_object)

[{"CODE_GENDER_M":1,"NAME_CONTRACT_TYPE_Cash_loans":1,"NAME_EDUCATION_TYPE_Lower_secondary":0,"EXT_SOURCE_3":1.0,"NAME_EDUCATION_TYPE_Higher_education":1,"NAME_EDUCATION_TYPE_Secondary_or_secondary_special":0}]


#### Dictionnaire multiple d'observations
- Ce fichier sera utilisé par les tests unitaires du script _test_2.py_
- Il est obtenu à partir des résultats TP (vrais positifs) des prédictions

In [15]:
nb_observations = 3

In [16]:
file_X_train = '../modeling/data/out/X_true_pos.csv'
file_X_sample = dir_test_data + 'X_sample.csv'
nb_file_lines = nb_observations + 1
! head -n $nb_file_lines $file_X_train > $file_X_sample
! wc -l $file_X_sample

4 ../test_api/data/X_sample.csv


In [20]:
df_X_sample = pd.read_csv(file_X_sample).drop('Unnamed: 0', axis='columns')
df_X_sample.head()

,CODE_GENDER_M,NAME_CONTRACT_TYPE_Cash_loans,NAME_EDUCATION_TYPE_Lower_secondary,EXT_SOURCE_3,NAME_EDUCATION_TYPE_Higher_education,NAME_EDUCATION_TYPE_Secondary_or_secondary_special
0,0,1,1,0.135951,0,0
1,0,1,1,0.244516,0,0
2,0,1,1,0.248536,0,0


In [21]:
str_features_values = df_X_sample.to_json(orient='records')
print(str_features_values)
with open(dir_test_data + '/dict_X_sample.json', 'w') as file_object:
    print(str_features_values, file=file_object)

[{"CODE_GENDER_M":0,"NAME_CONTRACT_TYPE_Cash_loans":1,"NAME_EDUCATION_TYPE_Lower_secondary":1,"EXT_SOURCE_3":0.1359510442,"NAME_EDUCATION_TYPE_Higher_education":0,"NAME_EDUCATION_TYPE_Secondary_or_secondary_special":0},{"CODE_GENDER_M":0,"NAME_CONTRACT_TYPE_Cash_loans":1,"NAME_EDUCATION_TYPE_Lower_secondary":1,"EXT_SOURCE_3":0.244516392,"NAME_EDUCATION_TYPE_Higher_education":0,"NAME_EDUCATION_TYPE_Secondary_or_secondary_special":0},{"CODE_GENDER_M":0,"NAME_CONTRACT_TYPE_Cash_loans":1,"NAME_EDUCATION_TYPE_Lower_secondary":1,"EXT_SOURCE_3":0.2485355573,"NAME_EDUCATION_TYPE_Higher_education":0,"NAME_EDUCATION_TYPE_Secondary_or_secondary_special":0}]


## Lancement du serveur

In [16]:
ip_host, port_staging = '0.0.0.0', '5677'
shell_command = 'mlflow models serve -m ' + model_path_staging + ' -p ' + port_staging + ' -h ' + ip_host
get_ipython().system_raw(shell_command + ' --no-conda &')          # runs model API in background

2024/06/03 22:58:25 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2024/06/03 22:58:25 INFO mlflow.pyfunc.backend: === Running command 'exec gunicorn --timeout=60 -b 0.0.0.0:5677 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2024-06-03 22:58:25 +0000] [268836] [INFO] Starting gunicorn 21.2.0
[2024-06-03 22:58:25 +0000] [268836] [INFO] Listening at: http://0.0.0.0:5677 (268836)
[2024-06-03 22:58:25 +0000] [268836] [INFO] Using worker: sync
[2024-06-03 22:58:25 +0000] [268837] [INFO] Booting worker with pid: 268837
2024/06/03 22:58:26 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.8.19`, differs from the version of Python that is currently running, `Python 3.10.12`, and may be incompatible


**Vérification d'exécution**

Il y a 2 processus qui tournent par serveur

In [17]:
! ps aux | grep "scoring_server" | grep -v "grep" | awk '{print $2, $15, $19}'

268836 0.0.0.0:5677 mlflow.pyfunc.scoring_server.wsgi:app
268837 0.0.0.0:5677 mlflow.pyfunc.scoring_server.wsgi:app


**Arrêt du serveur**

In [18]:
mask = ':' + port_staging
#! pkill -f "$mask"

## Configuration réseau
### Tunnel SSH
Sur le terminal [_Azure CLI_](https://portal.azure.com/#cloudshell/) :

In [27]:
shell_command = 'ssh -v -N -L ' + port_staging + ':localhost:' + port_staging + ' azureuser@' + ip_server
print(shell_command)

ssh -v -N -L 5677:localhost:5677 azureuser@13.92.86.145


### Aperture de ports
Sur le [portail _Azure_](https://portal.azure.com/), _Network Settings_ de la VM : _Create Inbound Port Rule_

In [123]:
print('Destination Port =', port_staging)

Destination Port = 5677


## Vérification d'accès distant

Demande par requête POST de prédiction de la cible pour une observation

In [29]:
url_staging = ip_server + ':' + port_staging + '/invocations'
print('URL Staging    -> http://' + url_staging)

URL Staging    -> http://13.92.86.145:5677/invocations


Données de l'observation en entrée 

In [32]:
str_features_values = df_observation.to_json(orient='split')
print(str_features_values)

{"columns":["CODE_GENDER_M","NAME_CONTRACT_TYPE_Cash_loans","NAME_EDUCATION_TYPE_Lower_secondary","EXT_SOURCE_3","NAME_EDUCATION_TYPE_Higher_education","NAME_EDUCATION_TYPE_Secondary_or_secondary_special"],"index":[0],"data":[[1,1,0,1.0,1,0]]}


In [33]:
str_data = '\'{"dataframe_split": ' + str_features_values + '}\' '
print(str_data)

'{"dataframe_split": {"columns":["CODE_GENDER_M","NAME_CONTRACT_TYPE_Cash_loans","NAME_EDUCATION_TYPE_Lower_secondary","EXT_SOURCE_3","NAME_EDUCATION_TYPE_Higher_education","NAME_EDUCATION_TYPE_Secondary_or_secondary_special"],"index":[0],"data":[[1,1,0,1.0,1,0]]}}' 


Copier cette ligne de commande Linux sur un terminal local :

In [34]:
print('curl -d' + str_data + '''-H 'Content-Type: application/json' -X POST ''' + url_staging)

curl -d'{"dataframe_split": {"columns":["CODE_GENDER_M","NAME_CONTRACT_TYPE_Cash_loans","NAME_EDUCATION_TYPE_Lower_secondary","EXT_SOURCE_3","NAME_EDUCATION_TYPE_Higher_education","NAME_EDUCATION_TYPE_Secondary_or_secondary_special"],"index":[0],"data":[[1,1,0,1.0,1,0]]}}' -H 'Content-Type: application/json' -X POST 13.92.86.145:5677/invocations


# Serveur de production

In [23]:
model_path_production = './production_model'
! mkdir -p $model_path_production

## Chargement du modèle

In [35]:
production_model = mlflow.pyfunc.load_model(model_path_production)
production_model

2024/06/03 15:05:28 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.8.19`, differs from the version of Python that is currently running, `Python 3.10.12`, and may be incompatible
/home/azureuser/environments_folder/my_env/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


mlflow.pyfunc.loaded_model:
  flavor: mlflow.sklearn

**Vérification de la signature**

La signature du modèle chargé indique les valeurs valides à lui fournir

Attributs

In [36]:
li_features = production_model.metadata.get_input_schema().input_names()
li_features

['CODE_GENDER_M',
 'NAME_CONTRACT_TYPE_Cash_loans',
 'NAME_EDUCATION_TYPE_Lower_secondary',
 'EXT_SOURCE_3',
 'NAME_EDUCATION_TYPE_Higher_education',
 'NAME_EDUCATION_TYPE_Secondary_or_secondary_special']

Cible

In [37]:
production_model.metadata.get_output_schema().input_names

<bound method Schema.input_names of ['TARGET': long (required)]>

**Vérification de prédiction**

On prédit la cible pour une observation

In [38]:
df_observation = pd.DataFrame(np.array([[1, 1, 0, 1.0, 1, 0]]).astype('int'), columns=li_features)
df_observation['EXT_SOURCE_3'] = df_observation['EXT_SOURCE_3'].astype('double')  # double~float64
production_model.predict(df_observation)

array([0])

## Lancement du serveur

In [21]:
ip_host, port_production = '0.0.0.0', '5678'
shell_command = 'mlflow models serve -m ' + model_path_production + ' -p ' + port_production + ' -h ' + ip_host
get_ipython().system_raw(shell_command + ' --no-conda &')          # runs model API in background

Traceback (most recent call last):
  File "/home/azureuser/environments_folder/my_env/bin/mlflow", line 8, in <module>
    sys.exit(cli())
  File "/home/azureuser/environments_folder/my_env/lib/python3.10/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/home/azureuser/environments_folder/my_env/lib/python3.10/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/home/azureuser/environments_folder/my_env/lib/python3.10/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/azureuser/environments_folder/my_env/lib/python3.10/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/azureuser/environments_folder/my_env/lib/python3.10/site-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/home/azureuser/environments_fold

**Vérification d'exécution**

Il y a 2 processus qui tournent par serveur

In [18]:
! ps aux | grep "scoring_server" | grep -v "grep" | awk '{print $2, $15, $19}'

268836 0.0.0.0:5677 mlflow.pyfunc.scoring_server.wsgi:app
268837 0.0.0.0:5677 mlflow.pyfunc.scoring_server.wsgi:app


**Arrêt du serveur**

In [44]:
mask = ':' + port_production
#! pkill -f "$mask"

## Configuration réseau
### Tunnel SSH
Sur le terminal [_Azure CLI_](https://portal.azure.com/#cloudshell/) :

In [56]:
shell_command = 'ssh -v -N -L ' + port_production + ':localhost:' + port_production + ' azureuser@' + ip_server
print(shell_command)

ssh -v -N -L 5678:localhost:5678 azureuser@13.92.86.145


### Aperture de ports
Sur le [portail _Azure_](https://portal.azure.com/), _Network Settings_ de la VM : _Create Inbound Port Rule_

In [123]:
print('Destination Port =', port_production)

Destination Port = 5677


## Vérification d'accès distant

Demande par requête POST de prédiction de la cible pour une observation

In [47]:
url_production = ip_server + ':' + port_production + '/invocations'
print('URL Production    -> http://' + url_production)

URL Production    -> http://13.92.86.145:5678/invocations


Données de l'observation en entrée 

In [48]:
str_features_values = df_observation.to_json(orient='split')
print(str_features_values)

{"columns":["CODE_GENDER_M","NAME_CONTRACT_TYPE_Cash_loans","NAME_EDUCATION_TYPE_Lower_secondary","EXT_SOURCE_3","NAME_EDUCATION_TYPE_Higher_education","NAME_EDUCATION_TYPE_Secondary_or_secondary_special"],"index":[0],"data":[[1,1,0,1.0,1,0]]}


In [49]:
str_data = '\'{"dataframe_split": ' + str_features_values + '}\' '
print(str_data)

'{"dataframe_split": {"columns":["CODE_GENDER_M","NAME_CONTRACT_TYPE_Cash_loans","NAME_EDUCATION_TYPE_Lower_secondary","EXT_SOURCE_3","NAME_EDUCATION_TYPE_Higher_education","NAME_EDUCATION_TYPE_Secondary_or_secondary_special"],"index":[0],"data":[[1,1,0,1.0,1,0]]}}' 


Copier cette ligne de commande Linux sur un terminal local :

In [50]:
print('curl -d' + str_data + '''-H 'Content-Type: application/json' -X POST ''' + url_production)

curl -d'{"dataframe_split": {"columns":["CODE_GENDER_M","NAME_CONTRACT_TYPE_Cash_loans","NAME_EDUCATION_TYPE_Lower_secondary","EXT_SOURCE_3","NAME_EDUCATION_TYPE_Higher_education","NAME_EDUCATION_TYPE_Secondary_or_secondary_special"],"index":[0],"data":[[1,1,0,1.0,1,0]]}}' -H 'Content-Type: application/json' -X POST 13.92.86.145:5678/invocations


<hr color="gold">

# Notes RAF

L’API de prédiction du score, déployée sur le cloud (lien vers l’API).

Un dossier, géré via un outil de versioning de code contenant :
- Le notebook ou code de la modélisation (du prétraitement à la prédiction),
- intégrant via MLFlow le tracking d’expérimentations et le stockage centralisé des modèles.
- Le code permettant de déployer le modèle sous forme d'API. Pour l’API,
    - un fichier introductif permettant de comprendre l'objectif du projet et
    - le découpage des dossiers, et
    - un fichier listant les packages utilisés
- Le tableau HTML d’analyse de data drift réalisé à partir d’evidently.



Aller vers >> [Sauvegarde N°1](#Restore_1) >> [Sauvegarde N°2](#Restore_2) >> [En cours](#Current_Cell)

<a name="Current_Cell"></a>
<hr color="red" size=2>

Aller vers >> [Modules&fonctions](#Fonctions) >> [Sauvegarde N°1](#Restore_1) >> [Sauvegarde N°2](#Restore_2) >> [En cours](#Current_Cell)
<hr color="red" size=2>

<a name="Current_Cell"></a>
<hr color="red" size=25>

# Fin du traitement

In [ ]:
assert(False) # prevents the execution of following cells

AssertionError: 

# Annexes

## Déploiement via MLFlow UI
La version du modèle peut être mise à disposition par _MLFlow UI_ afin d'être déployée 

**Instructions sous _MLFlow UI_ :**
1. click sur le _run_ du modèle à déployer
2. onglet "_Artifacts_", click sur le bouton "_Register_"
    - nommer le modèle à déployer "ml_model_to_deploy"
3. lancer le service du modèle 

## Référentiel d'évaluation
Remarque : le chiffre des dizaines a été ajouté aux références CE originales pour mieux les différencier

Définir la stratégie d’élaboration d’un modèle d’apprentissage supervisé et sélectionner et entraîner des modèles adaptés à une problématique métier afin de réaliser une analyse prédictive.
CE1 Les variables catégorielles identifiées ont été transformées en fonction du besoin (par exemple via OneHotEncoder ou TargetEncoder).

CE2 Vous avez a créé de nouvelles variables à partir de variables existantes.

CE3 Vous avez réalisé des transformations mathématiques lorsque c'est requis pour transformer les distributions de variables.

CE4 Vous avez normalisé les variables lorsque c'est requis.

CE5 Vous avez défini sa stratégie d’élaboration d’un modèle pour répondre à un besoin métier. Cela signifie dans ce projet que :

l’étudiant a présenté son approche méthodologique de modélisation dans son support de présentation pendant la soutenance et est capable de répondre à des questions à ce sujet, si elles lui sont posées.
CE6 Vous avez choisi la ou les variables cibles pertinentes.

CE7 Vous avez vérifié qu'il n’y a pas de problème de data leakage (c'est-à-dire, des variables trop corrélées à la variable cible et inconnues a priori dans les données en entrée du modèle).

CE8 Vous avez testé plusieurs algorithmes de façon cohérente, en partant des plus simples vers les plus complexes (au minimum un linéaire et un non linéaire).



Évaluer les performances des modèles d’apprentissage supervisé selon différents critères (scores, temps d'entraînement, etc.) en adaptant les paramètres afin de choisir le modèle le plus performant pour la problématique métier.

CE1 Vous avez choisi une métrique adaptée pour évaluer la performance d'un algorithme (par exemple : R2 ou RMSE en régression, accuracy ou AUC en classification, etc.). Dans le cadre de ce projet, cela signifie que :

Vous avez mis en oeuvre un score métier pour évaluer les modèles et optimiser les hyperparamètres, qui prend en compte les spécificités du contexte, en particulier le fait que le coût d’un faux négatif et d’un faux positif sont sensiblement différents.
CE2 Vous avez exploré d'autres indicateurs de performance que le score pour comprendre les résultats (coefficients des variables en fonction de la pénalisation, visualisation des erreurs en fonction des variables du modèle, temps de calcul...).

CE3 Vous avez séparé les données en train/test pour les évaluer de façon pertinente et détecter l'overfitting.

CE4 Vous avez mis en place un modèle simple de référence pour évaluer le pouvoir prédictif du modèle choisi (dummyRegressor ou dummyClassifier).

CE5 Vous avez pris en compte dans sa démarche de modélisation l'éventuel déséquilibre des classes (dans le cas d'une classification).

CE6 Vous avez optimisé les hyper-paramètres pertinents dans les différents algorithmes.

CE7 Vous avez mis en place une validation croisée (via GridsearchCV, RandomizedSearchCV ou équivalent) afin d’optimiser les hyperparamètres et comparer les modèles. Dans le cadre de ce projet :

une cross-validation du dataset train est réalisée ;
un premier test de différentes valeurs d’hyperparamètres est réalisé sur chaque algorithme testé, et affiné pour l’algorithme final choisi ;
tout projet présentant un score AUC anormalement élevé, démontrant de l’overfitting dans le GrisSearchCV, sera invalidé (il ne devrait pas être supérieur au meilleur de la compétition Kaggle : 0.82).
CE8 Vous avez présenté l'ensemble des résultats en allant des modèles les plus simples aux plus complexes. Vous avez justifié le choix final de l'algorithme et des hyperparamètres.

CE9 Vous avez réalisé l’analyse de l’importance des variables (feature importance) globale sur l’ensemble du jeu de données et locale sur chaque individu du jeu de données.



Définir et mettre en œuvre un pipeline d’entraînement des modèles, avec centralisation du stockage des modèles et formalisation des résultats et mesures des différentes expérimentations réalisées, afin d’industrialiser le projet de Machine Learning.
CE1 Vous avez mis en oeuvre un pipeline d’entraînement des modèles reproductible.

CE2 Vous avez sérialisé et stocké les modèles créés dans un registre centralisé afin de pouvoir facilement les réutiliser.

CE3 Vous avez formalisé des mesures et résultats de chaque expérimentation, afin de les analyser et de les comparer



Mettre en œuvre un logiciel de version de code afin d’assurer en continu l’intégration et la diffusion du modèle auprès de collaborateurs.
CE1 Vous avez créé un dossier contenant tous les scripts du projet dans un logiciel de version de code avec Git et l'a partagé avec Github.

CE2 Vous avez présenté un historique des modifications du projet qui affiche au moins trois versions distinctes, auxquelles il est possible d'accéder.

CE3 Vous avez tenu à jour et mis à disposition la liste des packages utilisés ainsi que leur numéro de version.

CE4 Vous avez rédigé un fichier introductif permettant de comprendre l'objectif du projet et le découpage des dossiers.

CE5 Vous avez commenté les scripts et les fonctions facilitant une réutilisation du travail par d'autres personnes et la collaboration.



Concevoir et assurer un déploiement continu d'un moteur d’inférence (modèle de prédiction encapsulé dans une API) sur une plateforme Cloud afin de permettre à des applications de réaliser des prédictions via une requête à l’API.

CE1 Vous avez défini et préparé un pipeline de déploiement continu.

CE2 Vous avez déployé le modèle de machine learning sous forme d'API (via Flask par exemple) et cette API renvoie bien une prédiction correspondant à une demande.

CE3 Vous avez mis en œuvre un pipeline de déploiement continu, afin de déployer l'API sur un serveur d'une plateforme Cloud.

CE4 Vous avez mis en oeuvre des tests unitaires automatisés (par exemple avec pyTest).

CE5 Vous avez réalisé l'API indépendamment de l'application qui utilise le résultat de la prédiction.



Définir et mettre en œuvre une stratégie de suivi de la performance d’un modèle en production et en assurer la maintenance afin de garantir dans le temps la production de prédictions performantes.

CE1 Vous avez défini une stratégie de suivi de la performance du modèle. Dans le cadre du projet :
choix de réaliser a priori cette analyse sur le dataset disponible : analyse de data drift entre le dataset train et le dataset test.

CE2 Vous avez réalisé un système de stockage d’événements relatifs aux prédictions réalisées par l’API et une gestion d’alerte en cas de dégradation significative de la performance. Dans le cadre du projet :
choix de réaliser a priori cette analyse analyse de data drift, via une simulation dans un notebook et création d’un tableau HTML d’analyse avec la librairie evidently.

CE3 Vous avez analysé la stabilité du modèle dans le temps et défini des actions d’amélioration de sa performance. Dans le cadre de ce projet :
analyse du tableau HTML evidently, et conclusion sur un éventuel data drift.